# Introduction to Athena and Glue - Free Cloud University

**Goal** The purpose of this lab is to expose you to interact with Athena and Glue using Python. By the end of this lab, you should be able to:
* Create a data catalog and database with Glue
* Create a data crawler with Glue
* Query datasets in S3 with Athena

Before you begin, ensure that you have Python 3 installed by running the code block below.

In [ ]:
!python3 --version

Run this code block to ensure that the boto3 library is installed.

In [ ]:
!pip install boto3

Run this code block to import the necessary packages. 

In [ ]:
#This module is necessary for interacting with AWS
import boto3

#We will be using these modules to create fake data
import time
from math import sin, cos
import random
import csv

import json

Running the code block below will set the region the boto3 library will create the resources.

In [ ]:
#We will be using US-East-1 as the default region
%env AWS_DEFAULT_REGION=us-east-1

Additionally, to complete the lab, you will need to come up with names for the resources. The account ID is accessible on your AWS console. You can name the rest of the resources whatever you want. 

In [ ]:
#Constants for code clarity
ACCOUNT_ID = None
IAM_GLUE_ROLE_NAME = None
S3_BUCKET_NAME = None
GLUE_DATABASE = None
GLUE_CRAWLER = None

Additionally, to complete this lab, you will need to run the following code block, which creates a file for you to upload to S3.

In [ ]:
#Creating a simple file to upload into S3
with open('sin_cos.csv','w') as file_to_write:
    csv_to_write = csv.writer(file_to_write)
    csv_to_write.writerow(['timestamp','sine','cosine','name'])
    for x in range(2000):
        current_timestamp = time.time() * 10000000
        csv_to_write.writerow([
            str(current_timestamp),
            sin(current_timestamp),
            cos(current_timestamp),
            random.choice(['Amanda','Becky','Cindy','Davis'])
        ])

Finally, before we can interact with all of the AWS resources, we need to create the boto3 clients. 

**Try It Out Yourself**: Fill in the missing `None` values with your AWS credentials and run the code block below.

In [ ]:
creds = {
    'aws_access_key_id' : None,
    'aws_access_secret_key' : None
}

athena = boto3.client(
    'athena',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_access_secret_key']
)
glue = boto3.client(
    'glue',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_access_secret_key']
)
s3 = boto3.resource(
    's3',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_access_secret_key']
)
iam = boto3.client(
    'iam',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_access_secret_key']
)

## Step 0: IAM
In order to run Glue processes, you need to create a specific IAM role with certain permissions. You can do this in the console, but for the sake of this tutorial, the role will be generated using the following blocks of code. First, the role needs to have a trust relationship that gives you access to Glue. That is set in the dictionary called `trust_relationship` below.

In [ ]:
#Let's first create a IAM role
trust_relationship = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "glue.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

glue_role = iam.create_role(
    Path='/',
    RoleName= IAM_GLUE_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(trust_relationship),
    Description='Simple policy for interacting with Glue'
)

role_arn = glue_role['Role']['Arn']
print(role_arn)

We also need to attach some policies to the role we created above. 

* AmazonS3FullAccess is a policy that gives the role access to reading and writing S3 objects.
* AmazonAthenaFullAccess is a policy that gives the role full access to Amazon Athena.
* AWSGlueConsoleSageMakerNotebookFullAccess is a policy that gives the role access to methods that interact with Glue and SageMaker.
* AWSGlueConsoleFullAccess is a policy that gives the role access to Glue Console methods.
* CloudWatchFullAccess is a policy that allows the role to access Cloud Watch results.
* AWSCloudFormationReadOnlyAccess is a policy that allows the role to access Cloud Formation methods.

We need all of these policies in order for this lab to work.

In [ ]:
policies = [
    'AmazonS3FullAccess',
    'AmazonAthenaFullAccess',
    'AWSGlueConsoleSageMakerNotebookFullAccess',
    'AWSGlueConsoleFullAccess',
    'CloudWatchFullAccess',
    'AWSCloudFormationReadOnlyAccess'    
]
for policy in policies:
    iam.attach_role_policy(
        RoleName=IAM_GLUE_ROLE_NAME,
        PolicyArn= f'arn:aws:iam::aws:policy/{policy}'
    )

## Step 1A: S3

We need to create a source for Glue to crawl and Athena to query. So create a bucket and upload a the file below by running the following code block.

In [ ]:
#First, let's create an S3 bucket upload the CSV file
bucket = s3.Bucket(S3_BUCKET_NAME)
bucket.create()

filename = 'sin_cos.csv'
with open(filename, 'rb') as data:
    bucket.upload_fileobj(data, 'sin_cos.csv')

## Step 2: Glue

Now that we have the file loaded into S3, we will now process the data using Glue. To do that, we must first create a database and a crawler to read the data in S3.

Let's first create the database using the `create_database` method with the `glue` client. There are two required parameters: the catalog id (which in this case is your AWS Account ID) and `DatabaseInput`, which is a dictionary that contains the name of the database you want to create. 

**Try It Out**: Run the code sample below to create a Glue database.

In [ ]:
#Let's now create a database using Glue
database = glue.create_database(
    CatalogId=ACCOUNT_ID,
    DatabaseInput={
        'Name': GLUE_DATABASE
    }
)

Next, we will create a crawler using the `create_crawler` method using the `glue` client. It takes a couple of parameters. First it takes the name you want to assign to the crawler. You must also provide the ARN of the policy we created above. Additionally, you will need to provide the name of the database we just created. You will also provide targets, which are the places Glue will read to find data. For this example, we will simply provide S3 file paths for Glue to read. Also, we will provide a schema change policy and configuration parameters.

**Note**: When supplying the S3 targets to Glue, the paths must be folders and not files.

**Try It Out**: Run the code sample below to create a crawler.

In [ ]:
#Let's now crawl through the file uploaded to S3
config = {
    "Version" : 1.0,
    "CrawlerOutput" : {
        "Partitions" : {
            "AddOrUpdateBehavior" : "InheritFromTable"
        }
    }
}
crawler = glue.create_crawler(
    Name = GLUE_CRAWLER,
    Role = role_arn,
    DatabaseName = GLUE_DATABASE,
    Targets = {
        'S3Targets': [
            {'Path': f's3://{S3_BUCKET_NAME}'}
        ]
    },
    SchemaChangePolicy={
        'UpdateBehavior': 'UPDATE_IN_DATABASE',
        'DeleteBehavior': 'DELETE_FROM_DATABASE'
    },
    Configuration=json.dumps(config)
)

Let's run the crawler we just created. To do this, you must use the `start_crawler` method providing the name of the crawler. To check on the crawler's status, use the `get_crawler` method, also providing the name of the crawler.

**Try It Out**: Run the code block below. It will start the crawler you just created and check on its status every ten seconds. It will take roughly a minute for this code to run.

In [ ]:
#Start the crawler
glue.start_crawler(
    Name=GLUE_CRAWLER
)

#Check on the status of the crawler
crawl_status = glue.get_crawler(
    Name=GLUE_CRAWLER
)

while crawl_status['Crawler'].get('State') in ['RUNNING','STOPPING']:
    print('Getting crawl status...')
    time.sleep(10)
    crawl_status = glue.get_crawler(
        Name=GLUE_CRAWLER
    )

Verify that your code ran correctly by running the block below. If not, then check your AWS credentials and other portions of the code for accuracy.

In [ ]:
#Check to see if crawler errored out in any way
if crawl_status['Crawler'].get('LastCrawl',{}).get('ErrorMessage') != None:
    print("There was an error in the crawling")
else:
    print("Everything is okay.")

Because the crawler creates a table upon crawling the S3 buckets, we need to actually find the table created. The code block below usses the `get_tables` method and searches for a table with a `StorageDescriptor.Location` that contains the S3 bucket we created above. This should work since we just created the bucket. But if you used the bucket created in this lab for other Glue tables, then this might not work correctly.

In [ ]:
#Search for tables and grab table name
glue_tables = glue.get_tables(
    CatalogId=ACCOUNT_ID,
    DatabaseName=GLUE_DATABASE
)
table_name = ''
for table in glue_tables['TableList']:
    if table.get('StorageDescriptor',{}).get('Location') == f's3://{S3_BUCKET_NAME}/':
        table_name = table.get('Name')
print(f'The table created by the crawler is named: {table_name}')

## Step 3: Athena

Finally, we will query the data we crawled by using Athena. Simply call the `start_query_execution` method to query the data. The first argument you will provide is the `QueryString`, which is a SQL query. Secondly, you will provide a dictionary containing the name of the Glue database created above. Finally, you will then need to point the output of the query to an S3 bucket by using the `ResultConfiguration` parameter.

After we start the query execution, we can get the status of the execution by using the `get_query_execution` method by retrieving the `QueryExecutionId` from the result of the `start_query_execution` method.

**Try It Out**: Run the code block below in order to query the table the crawler created. Wait until the code block finishes.

In [ ]:
#Start a query execution
query_data = athena.start_query_execution(
    QueryString = f'SELECT COUNT(*), name FROM {table_name} GROUP BY name',
    QueryExecutionContext = {
        'Database': GLUE_DATABASE
    }, 
    ResultConfiguration = {'OutputLocation': f's3://{S3_BUCKET_NAME}'}
)

#Get query execution data
query_exec_results = athena.get_query_execution(
    QueryExecutionId = query_data['QueryExecutionId']
)
while query_exec_results['QueryExecution']['Status'].get('State') in ['QUEUED','RUNNING']:
    time.sleep(2)
    print('Getting query execution')
    query_exec_results = athena.get_query_execution(
        QueryExecutionId = query_data['QueryExecutionId']
    )

Now that the code block is finished executing, you can see the results by running the `get_query_results` method. You can also download the file object created from S3.

In [ ]:
#Get query results
query_results = athena.get_query_results(
    QueryExecutionId = query_data['QueryExecutionId']
)

query_results